# Download weight

In [ ]:
# Delete all files and folders in the current directory. Use with caution!
import shutil
shutil.rmtree('./', ignore_errors=True)

In [ ]:
!git clone https://github.com/BinhPQ2/Han_Nom_OCR.git .
!git clone https://github.com/THU-MIG/yolov10.git

Cloning into '.'...
remote: Enumerating objects: 156, done.
remote: Counting objects: 100% (156/156), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 156 (delta 18), reused 152 (delta 17), pack-reused 0 (from 0)
Receiving objects: 100% (156/156), 8.85 MiB | 38.41 MiB/s, done.
Resolving deltas: 100% (18/18), done.
Cloning into 'yolov10'...
remote: Enumerating objects: 20329, done.
remote: Counting objects: 100% (2443/2443), done.
remote: Compressing objects: 100% (247/247), done.
remote: Total 20329 (delta 2315), reused 2196 (delta 2196), pack-reused 17886 (from 1)
Receiving objects: 100% (20329/20329), 11.14 MiB | 25.76 MiB/s, done.
Resolving deltas: 100% (14335/14335), done.


In [ ]:
!gdown --id 11VxsyUVmdH8yD3MtQT-t4I2q9YIvkp3w -O file.zip
!unzip -q file.zip

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=11VxsyUVmdH8yD3MtQT-t4I2q9YIvkp3w
From (redirected): https://drive.google.com/uc?id=11VxsyUVmdH8yD3MtQT-t4I2q9YIvkp3w&confirm=t&uuid=442f34fb-6724-42f6-90fd-18f3fc8f543d
To: /content/file.zip
100% 262M/262M [00:01<00:00, 194MB/s]


# Detection

In [ ]:
!pip install -q -r requirements.txt
!pip install -q -e ./yolov10/
# !pip install supervision -q # already included in the requirement.txt
# !pip install ultralytics -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 102.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 858.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.2/426.2 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62

In [ ]:
!pip show ultralytics

Name: ultralytics
Version: 8.1.34
Summary: Ultralytics YOLOv8 for SOTA object detection, multi-object tracking, instance segmentation, pose estimation and image classification.
Home-page: 
Author: Glenn Jocher, Ayush Chaurasia, Jing Qiu
Author-email: 
License: AGPL-3.0
Location: /usr/local/lib/python3.10/dist-packages
Editable project location: /content/yolov10
Requires: matplotlib, opencv-python, pandas, pillow, psutil, py-cpuinfo, pyyaml, requests, scipy, seaborn, thop, torch, torchvision, tqdm
Required-by: 


In [ ]:
import os
import random
import numpy as np
from tqdm import tqdm
import pickle
import cv2
import supervision as sv
import matplotlib.pyplot as plt
from ultralytics import YOLOv10 # rerun session if it has error in this library

In [ ]:
def save_crop(result, save_path):
    os.makedirs(save_path, exist_ok=True)

    conf_s = result.boxes.conf.detach().cpu().numpy()
    xyxy_s = result.boxes.xyxy.detach().cpu().numpy()
    idx_s = [i for i in range(len(conf_s))]
    img_name = os.path.basename(result.path)
    img = result.orig_img
    for idx, conf, xyxy in zip(idx_s, conf_s, xyxy_s):
        # name = f"oder_{idx}__conf_{conf}__name_{img_name}"
        name = img_name

        height=result.orig_img.shape[0]
        width=result.orig_img.shape[1]

        height_extend_ratio = 0.05
        width_extend_ratio = 0.1

        x0, y0, x1, y1 = xyxy.tolist()
        x0 = int(max(0, x0 - width_extend_ratio*abs(x1 - x0)))
        x1 = int(min(width, x1 + width_extend_ratio*abs(x1 - x0)))
        y0 = int(max(0, y0 - height_extend_ratio*abs(y1 - y0)))
        y1 = int(min(height, y1 + height_extend_ratio*abs(y1 - y0)))
        crop_image = img[y0:y1, x0:x1]

        cv2.imwrite(os.path.join(save_path, name), crop_image)

In [ ]:
input_image_folder = "./input_image"
save_path = "./output_detection"
model = YOLOv10('./weight/detection_yolov10.pt')

results = model.predict(source=input_image_folder, conf=0.5, stream=True, save_txt=True)
for idx, result in enumerate(results):
    save_crop(result, save_path)


image 1/23 /content/input_image/02b02efc5d6b4579a0cae31e1e362555.jpg: 640x480 1 couplet, 65.2ms
image 2/23 /content/input_image/0ff04276ca994b12a50f51f88a9373fc.jpg: 640x352 1 couplet, 73.9ms
image 3/23 /content/input_image/1a2291615b4e4c288a152b4886ac300e.jpg: 640x480 1 couplet, 30.1ms
image 4/23 /content/input_image/1a5634d0281b4b1aac6e616826c30d4b.jpg: 640x320 2 couplets, 74.2ms
image 5/23 /content/input_image/1a7296dbb22748e49b656df5cc6743b5.jpg: 640x288 2 couplets, 84.4ms
image 6/23 /content/input_image/1b094caec3984b8eb1c929b60a8d2b08.jpg: 640x480 4 couplets, 29.9ms
image 7/23 /content/input_image/1b648fd456b54ea5a6a4ff1222b95bad.jpg: 640x384 1 couplet, 68.4ms
image 8/23 /content/input_image/1b7b10992eb040d19fcfcadb745d1aa0.jpg: 640x256 1 couplet, 63.2ms
image 9/23 /content/input_image/1be6c13d483c41e6809a843fb9f1e913.jpg: 640x480 1 couplet, 30.0ms
image 10/23 /content/input_image/1bf527e50d2447098ef82c510e5dab4d.jpg: 640x288 1 couplet, 23.7ms
image 11/23 /content/input_image/1c

# Recognition

In [ ]:
!pip install -q paddleocr --upgrade
!pip install -q paddlepaddle-gpu --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.5/407.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.9/758.9 MB 2.1 MB/s eta 0:00:00


In [15]:
!python3 /content/recognition/ppocr/tools/infer/predict_rec.py --image_dir="./output_detection" \
                                                    --rec_model_dir='./weight/recognition_PPOCRLABEL' \
                                                    --rec_char_dict_path='/content/recognition/ppocr/utils/ppocr_keys_v1.txt' \
                                                    # --use_gpu=False \

[2024/08/27 17:00:28] ppocr WARNING: The first GPU is used for inference by default, GPU ID: 0
[2024/08/27 17:00:31] ppocr INFO: In PP-OCRv3, rec_image_shape parameter defaults to '3, 48, 320', if you are using recognition model with PP-OCRv2 or an older version, please set --rec_image_shape='3,32,320
[2024/08/27 17:00:33] ppocr INFO: Predicts of ./output_detection/02b02efc5d6b4579a0cae31e1e362555.jpg:('', 0.0)
[2024/08/27 17:00:33] ppocr INFO: Predicts of ./output_detection/0ff04276ca994b12a50f51f88a9373fc.jpg:('', 0.0)
[2024/08/27 17:00:33] ppocr INFO: Predicts of ./output_detection/1a2291615b4e4c288a152b4886ac300e.jpg:('', 0.0)
[2024/08/27 17:00:33] ppocr INFO: Predicts of ./output_detection/1a5634d0281b4b1aac6e616826c30d4b.jpg:('', 0.0)
[2024/08/27 17:00:33] ppocr INFO: Predicts of ./output_detection/1a7296dbb22748e49b656df5cc6743b5.jpg:('', 0.0)
[2024/08/27 17:00:33] ppocr INFO: Predicts of ./output_detection/1b094caec3984b8eb1c929b60a8d2b08.jpg:('', 0.0)
[2024/08/27 17:00:33] ppo